<a href="https://colab.research.google.com/github/RickFaint/varioususefulbatchfiles/blob/main/locationImport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This section allows you to upload your files

In [ ]:
import ipywidgets as widgets
from IPython.display import display

## run this cell by pressing the play button and then select the file from your drive
## file should have two columns the address and the postcode

## this code just creates some boxes to put values in. If the boxes are not populated then it will error




text_descriptionLocation = widgets.HTML(value=" Select a location file CSV file to upload .")
text_descriptionTown = widgets.HTML(value=" Select a town file to upload")

uploadLocation = widgets.FileUpload(

    accept='.csv',  # Specify the accepted file type (optional)
    multiple=False   # Set to True if you want to allow multiple files to be uploaded
)

uploadTown = widgets.FileUpload(

    accept='.csv',  # Specify the accepted file type (optional)
    multiple=False   # Set to True if you want to allow multiple files to be uploaded
)
display(uploadLocation, text_descriptionLocation)
display(uploadTown, text_descriptionTown)


FileUpload(value={}, accept='.csv', description='Upload')

HTML(value=' Select a location file CSV file to upload .')

FileUpload(value={}, accept='.csv', description='Upload')

HTML(value=' Select a town file to upload')

#This section processes your location file

In [108]:
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install geopy
import io
from google.colab import drive
import pandas as pd #import panda library
import re #import regex library
import seaborn as sns #import seaborn
import matplotlib.pyplot as plt #import matplotlib
from geopy.geocoders import Nominatim

#geocoding agent
geolocator = Nominatim(user_agent="postcode_locator",  timeout=30)

def get_lat_long(postcode):
#calls geocoder
    location = geolocator.geocode(postcode)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

def clean_address(address):
#removes punctuation from address and replaces with space
    if isinstance(address, str):
        return re.sub(r'\W+', ' ', address)
    else:
        return ''

def clean_and_replace(address):
#method to remove common strings . Checks in the word_replacements dictionary
    if isinstance(address, str) and len(address) > 25:
        for word, replacement in word_replacements.items():
            address = address.replace(word, replacement)
    return address

# used to remove common words if the string is to long. We should add new ones as we use this sheet
word_replacements = {
    'Estate': 'Est',
    'Industrial': 'Ind',
    '  ': ' ',
    'Limited': 'Ltd',
    'Road': 'Rd',
    'Avenue': 'Ave',

}

# Function to extract outward code using regex
def extract_outward_code(postal_code):
    match = re.match(r'^[A-Z]{1,2}\d{1,2}[A-Z]?', postal_code)
    if match:
        return match.group()
    else:
        return None

drive.mount('/content/drive') # this mounts your google drive however you can just upload from your C drive or other network location but the results are saved to a temp file on your google drive

uploadedLocation_file = list(uploadLocation.value.values())[0] #gets the file you have uploaded


# Get the file content as bytes
location_content = uploadedLocation_file['content']

addressdf = pd.read_csv(io.BytesIO(location_content)) # upload file as a raw string. files are uploaded into a panda.

addressdf.columns = ['extenalcode', 'name', 'address1','address2','town','county','postcode'] #rename columns so that it won't error if columns named differently



addressdf[['latitude', 'longitude']] = addressdf['postcode'].apply(get_lat_long).apply(pd.Series) #applies lat and long

#cleans addresses and converts to upper case .
addressdf['cleaned_name'] = addressdf['name'].apply(clean_address).str.upper()
addressdf['cleaned_address1'] = addressdf['address1'].apply(clean_address).str.upper()
addressdf['cleaned_address2'] = addressdf['address2'].apply(clean_address).str.upper()

#shorten addresses if more than 25 characters. It does this by removing common strings. Once that is all done it gets the first 25
addressdf['cleaned_name'] = addressdf['cleaned_name'].apply(clean_and_replace)[:25].str.upper()
addressdf['cleaned_address1'] = addressdf['cleaned_address1'].apply(clean_and_replace)[:25].str.upper()
addressdf['cleaned_address2'] = addressdf['cleaned_address2'].apply(clean_and_replace)[:25].str.upper()

#add operations description

addressdf['ops_description'] = addressdf['cleaned_name'] + " " + addressdf["postcode"].str.upper().str[:25]

#get outer postcodes and also the first 2 char of the postcode
addressdf['outward_code'] = addressdf['postcode'].apply(extract_outward_code)
addressdf['shortoutward_code'] = addressdf['postcode'].str[:2]

#create town pc for match to town file
addressdf['townpc'] = addressdf['town'] + addressdf["outward_code"]
addressdf['townpc'] = addressdf['townpc'].str.upper()
addressdf['shorttownpc'] = addressdf['town'] + addressdf["shortoutward_code"]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#This section processes your town file and creates + exports the finished file

In [ ]:


uploadedTown_file = list(uploadTown.value.values())[0] # get the town file


# Get the file content as bytes
town_content = uploadedTown_file['content']

towndf = pd.read_csv(io.BytesIO(town_content), encoding='cp1252') # upload file as a raw string. files are uploaded into a panda.

# new columns used for merging the location and town dataframe
towndf['tf_townpc'] = towndf['Town'] + towndf["Dft PC"]
towndf['tf_townpc'] = towndf['tf_townpc'].str.upper()

# rename the columns
towndf.columns = ['TFTown','TFCounty','TownDFPC','TownTA','TFCountry','tf_townpc']
towndf['tfshorttownpc'] = towndf['TFTown'] + towndf["TownDFPC"].str[:2]
towndf['tfshortpc'] =  towndf["TownDFPC"].str[:2]

# merge the dataframes on the town and pc

merged_df = addressdf.merge(towndf, left_on='townpc', right_on='tf_townpc', how='left')


# Dictionaries used for mapping the towns and counties. Hopefully most of the work will be done by the merge above
# but if the towns they provide dont exist we just use the ones in the town file . initially we map on the full outer, if that fails we use the 1st two chars only
short_town_pc_mapping = towndf.set_index('tfshorttownpc')['TFTown'].to_dict()
short_county_pc_mapping = towndf.set_index('tfshorttownpc')['TFCounty'].to_dict()
town_pc_mapping = towndf.set_index('TownDFPC')['TFTown'].to_dict()
county_pc_mapping = towndf.set_index('TownDFPC')['TFCounty'].to_dict()



# Create the 'final_town' column using the mapping the first section tries and uses the town they provided
merged_df['final_townW'] = merged_df['tfshorttownpc'].map(short_town_pc_mapping)
merged_df['final_countyW'] = merged_df['tfshorttownpc'].map(short_county_pc_mapping)


# if we cannot use the ones they provided then use the ones in the town and county files
merged_df['final_town'] = merged_df['outward_code'].map(town_pc_mapping)
merged_df['final_county'] = merged_df['outward_code'].map(county_pc_mapping)



# export format
# columns for the export
columns = [
    'Location Code', 'Location Name', 'Address Line 1', 'Address Line 2',
    'Town', 'County', 'Postcode', 'Phone No.', 'Fax No.', 'Contact Name',
    'Operational Description', 'Imported GTS Codes', 'Open Time', 'Close Time',
    'Special Instruction Code 1', 'Special Instruction Text 1',
    'Special Instruction Code 2', 'Special Instruction Text 2',
    'Special Instruction Code 3', 'Special Instruction Text 3',
    'Special Instruction Code 4', 'Special Instruction Text 4',
    'Special Instruction Code 5', 'Special Instruction Text 5',
    'Latitude', 'Longitude', 'Email Address', 'Delivery Location Group',
    'Ring Fence Distance', 'Restricted', 'Pre-Advice (in minutes)',
    'Email Address 1', 'Email Address 2', 'Email Address 3',
    'Email Address 4', 'Email Address 5'
]


# Create an empty DataFrame with the specified columns
export_dfw = pd.DataFrame(columns=columns)

# create two dicts holding the source columns and the destination columns

source_columns = ['extenalcode',  'cleaned_name', 'cleaned_address1',
       'cleaned_address2', 'final_town', 'final_county', 'postcode', 'ops_description','latitude','longitude']
destination_columns = ['Location Code', 'Location Name', 'Address Line 1', 'Address Line 2',
    'Town', 'County', 'Postcode', 'Operational Description', 'Latitude', 'Longitude']

# Merge into the empty dataframe which is in the correct format
for source_col, dest_col in zip(source_columns, destination_columns):
    export_dfw[dest_col] = merged_df[source_col]

#corrects any final missing towns and counties based on first two char of postcode

for index, row in export_dfw.iterrows():
    if pd.isnull(row['Town']):
        postcode_prefix = row['Postcode'][:2]
        matching_rows = towndf[towndf['tfshortpc'].notnull() & towndf['tfshortpc'].str.contains(postcode_prefix)]
        if not matching_rows.empty:
            town_value = matching_rows.iloc[0]['TFTown'].strip()
            county_value = matching_rows.iloc[0]['TFCounty'].strip()
            export_dfw.at[index, 'Town'] = town_value
            export_dfw.at[index, 'County'] = county_value

#fill in some default data. Not sure if we should fill in more with oher data

export_dfw['Ring Fence Distance'] = export_dfw['Ring Fence Distance'].fillna(250)
export_dfw['Phone No.'] = export_dfw['Phone No.'].fillna('01429 265674')
export_dfw['Email Address'] = export_dfw['Email Address'].fillna('Mike@gmail.com')
export_dfw['Contact Name'] = export_dfw['Contact Name'].fillna('Mike')

# replace NA values with blank data
export_df = export_dfw.fillna('')

# saves the file on your google drive

export_df.to_csv(r"/content/locationfile.csv",index=False)




   Location Code              Location Name Address Line 1 Address Line 2  \
0         ROCBRI                   ROCKWOOL                                 
1         NESMEL                 NESTLE MCE                                 
2         NESBUX                 NESTLE WSB                                 
3         NESKEG                 NESTLE EMG                                 
4         NESHAM                 NESTLE HH2                                 
5         NESNEW                 NESTLE NPO                                 
6         NESNAR                 NESTLE PGW                                 
7         BELBIR   BELGRADE INSULATIONS LTD                                 
8         SIGOLD             SIG INSULATION                                 
9         GALWES    GALAXY INSTN DRY LINING                                 
10        MAYBUR                    MAYPLAS                                 
11        AVOAVO            AVONMOUTH CO OP                                 

# Mapping

In [110]:
!pip install folium
import pandas as pd
import folium

# get the lats and longs from the files created earlier
export_df['Latitude'] = pd.to_numeric(export_df['Latitude'])
export_df['Longitude'] = pd.to_numeric(export_df['Longitude'])
#drop any nulls as they cannot be map
export_df = export_df.dropna(subset=['Latitude'])
export_df = export_df.dropna(subset=['Longitude'])

# Create a map centered at the mean of latitudes and longitudes
center_lat = export_df['Latitude'].mean()
center_lon = export_df['Longitude'].mean()
#create the map
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# Add markers for locations from the DataFrame
for index, row in export_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Location Name'] +" " + row['Postcode']
    ).add_to(my_map)

# Save the map to an HTML file on your Google drive. You need to download and open. it is an interactive map
my_map.save('locations_map.html')

print("Map created and saved as 'locations_map.html'")


Map created and saved as 'locations_map.html'
